In [8]:
from selenium import webdriver
import pandas as pd

In [9]:
driver = webdriver.Chrome()

In [10]:
base_url = 'https://www.olx.com.eg/en/properties/apartments-duplex-for-sale'
driver.get(base_url)

In [11]:
last_page = driver.find_elements_by_css_selector('div.pager span[class="item fleft"]')
last_page_number = int(last_page[-1].find_elements_by_css_selector('a span')[0].text)
last_page_number

500

In [12]:
ads_link = []
def get_ads_link(links_list,driver):
    ads = driver.find_elements_by_css_selector('div.ads--list div.ads__item div.ads__item__info')
    for ad in ads:
        link = ad.find_elements_by_css_selector('a')[0].get_attribute('href')
        links_list.append(link[:23]+'en/'+link[23:])

In [13]:
for page in range(1,last_page_number+1):
    driver.get(f"{base_url}/?page={str(page)}")
    get_ads_link(ads_link,driver)

In [14]:
def get_ad_info(driver):
    df = pd.DataFrame()
    aspects = {}
    
    
    apartment_features = driver.find_elements_by_css_selector('div.descriptioncontent table.details tbody tr td table tbody tr')
    for features in apartment_features:
        feature = features.find_elements_by_css_selector('th')[0].text
        value = features.find_elements_by_css_selector('td')[0].text
        aspects[feature] = value
    
    if len(driver.find_elements_by_css_selector('div.offerbox div.pricelabel')) > 0:
        price = driver.find_elements_by_css_selector('div.offerbox div.pricelabel')[0].text.split()[0].replace(',','')
        aspects['Price'] = price
    

    if len(driver.find_elements_by_css_selector('a#showMap span[class="block clr"] span[class="block address fleft marginleft15"] span')) > 0:
        location = driver.find_elements_by_css_selector('a#showMap span[class="block clr"] span[class="block address fleft marginleft15"] span')[0].text
        aspects['Location'] = location
    
    df = df.append(aspects,ignore_index = True)
    return df

In [15]:
data = pd.DataFrame()
for link in ads_link:
    driver.get(link)
    data = data.append(get_ad_info(driver),ignore_index = True)

In [16]:
data.head()

,Amenities,Area (m²),Bathrooms,Bedrooms,Furnished,Level,Location,Payment Option,Price,Type,Down Payment,Compound
0,Balcony\nCovered Parking\nPets Allowed\nSecuri...,135,2,3,No,2,Sadat، Monufia,Cash or Installment,513000,Apartment,NaN,NaN
1,Balcony\nCovered Parking\nSecurity\nPool\nElec...,97,2,1,NaN,2,Sheikh Zayed، Giza,Installment,1950000,Apartment,"1,265,000",NaN
2,Balcony\nBuilt in Kitchen Appliances\nCentral ...,175,3,4,NaN,Ground,Shorouk City، Cairo,NaN,3500000,Duplex,175,NaN
3,Balcony\nBuilt in Kitchen Appliances\nCentral ...,85,2,2,NaN,9,New Capital City، Cairo,Cash or Installment,1850000,Duplex,"185,000",NaN
4,,300,3,3,No,3,Shorouk City، Cairo,Cash or Installment,2500000,Penthouse,NaN,Granda


In [17]:
data.shape

(22498, 12)

In [18]:
driver.quit()

In [19]:
data.to_csv('apratments_data.csv')